In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
dat = pd.read_csv("./drive/MyDrive/SMCAL/starD/stard.csv")
print(dat.shape)

(319, 308)


In [3]:
print(dat.head())

   ID   tx  qctot  SEX  ...  grseb_rate  cgi_i_rate  REMSN  SETTING
0   3  BUP      9    1  ...   -0.250000    0.250000      0        1
1  24  BUP      1    2  ...    0.428571   -0.142857      1        0
2  27  BUP      5    1  ...    0.142857   -0.142857      0        1
3  28  SER     13    2  ...    0.750000    0.000000      0        1
4  50  BUP     20    1  ...    0.142857    0.142857      0        1

[5 rows x 308 columns]


In [54]:
#STARD
#Date: 12/29/20

import numpy as np 

#sigmoid function
def f(x):
  return (1/(1+np.exp(-x)))


#the derivative of f
def df(x):
  x = np.float32(x)
  return (1/(np.exp(-x)+np.exp(x)+2))


#calculate gradient
def g(Xb,W1,row,a,n):
  Xb = np.repeat(Xb[...,np.newaxis], n, axis=1)
  Xb = Xb-Xb.T
  dXbeta=df(Xb*a)
  xk_xk = np.repeat(row[...,np.newaxis], n, axis=1)
  xk_xk = xk_xk-xk_xk.T
  return (-a*sum(sum(W1*dXbeta*xk_xk))/(n*(n-1)))


#soft-thresholding operator
def Soft_thres(x,lbd_t):
  if x>lbd_t:
    return (x-lbd_t)
  elif x+lbd_t<0:
    return (x+lbd_t)
  else:
    return 0


#grid search for intercept
def h(W, Xbeta, t):
  return sum(W[Xbeta>t])


#grid search for alpha
def alpha_grid(x):
  return 2.0**(x/2-0.0) 


#grid search for lambda
def lambda_grid(x):
  return (x+2)/3 


#IPW
def ipw(Y1, A1, xbeta1, c1):
  return np.mean(Y1[A1==(xbeta1>c1)])



In [ ]:
#determine the most useful coordinate

d=305
n=319
X = dat.values[:,3:308].T
A_raw = dat.values[:,1]
A = np.zeros(n)
A[A_raw=="BUP"]=1

Y=-dat.values[:,2]
v=np.mean(Y[A==0])
pi = sum(A)/n
W=(Y-v)*(A-pi)/(pi*(1-pi))

score = np.zeros((2,d))
for j0 in range(d):
  print(j0)
  beta = np.zeros(d)
  beta[j0] = 1
  #estimate threshold
  Xbeta=np.matmul(beta,X)
  candidate = list(Xbeta)
  candidate.append(min(Xbeta)-0.01)
  c_cv_objective = list(map(lambda x: h(W, Xbeta, x), candidate)) 
  max_index = np.argmax(c_cv_objective)
  b0= candidate[max_index]
  sample1=set(np.where(Xbeta>b0)[0])
  sample0=set(np.where(Xbeta<=b0)[0])
  bup=set(np.where(A==1)[0])
  ser=set(np.where(A==0)[0])
  valid_samples = set((sample1&bup)|(sample0&ser))
  bootmean=np.zeros(1000)
  bootn=1000
  for i in range(1000): 
      boot= np.random.choice([i for i in range(n)],bootn)
      boot_valid = list(map(lambda x: x in valid_samples, boot))
      IPW=Y[boot]*boot_valid/(A[boot]*pi+(1-A[boot])*(1-pi))
      bootmean[i]=sum(IPW)/bootn
  score[0,j0]=np.mean(bootmean)

for j0 in range(305):
  print(j0)
  beta = np.zeros(d)
  beta[j0] = -1
  #estimate threshold
  Xbeta=np.matmul(beta,X)
  candidate = list(Xbeta)
  candidate.append(min(Xbeta)-0.01)
  c_cv_objective = list(map(lambda x: h(W, Xbeta, x), candidate)) 
  max_index = np.argmax(c_cv_objective)
  b0= candidate[max_index]
  sample1=set(np.where(Xbeta>b0)[0])
  sample0=set(np.where(Xbeta<=b0)[0])
  bup=set(np.where(A==1)[0])
  ser=set(np.where(A==0)[0])
  valid_samples = set((sample1&bup)|(sample0&ser))
  bootmean=np.zeros(1000)
  bootn=1000
  for i in range(1000): 
      boot= np.random.choice([i for i in range(n)],bootn)
      boot_valid = list(map(lambda x: x in valid_samples, boot))
      IPW=Y[boot]*boot_valid/(A[boot]*pi+(1-A[boot])*(1-pi))
      bootmean[i]=sum(IPW)/bootn
  score[1,j0]=np.mean(bootmean)



In [11]:
#determine the most useful coordinate
print(score)
print(np.argmax(score))

[[-10.32446723 -10.50559817  -9.79539759 -10.52243407 -10.5223034
  -10.52838361 -10.51061378 -10.52324695 -10.53564951 -10.53451572
  -10.51363659  -9.85552492 -11.13038982 -10.85261457 -10.51227988
  -10.85312632  -9.0250705  -10.51054075 -10.42336914 -10.50787153
  -11.46984722 -11.06652797 -10.49511537 -10.58312115 -11.06617539
  -10.51382299 -10.52481312 -10.64788035  -9.56558102 -10.53738672
  -10.53184649 -10.63264661 -10.46023163 -10.51060417 -10.4315377
  -10.68338173  -9.70142569 -10.53982918 -10.1111466  -10.51631158
  -10.54178141 -10.52582201  -9.93269047 -10.58969968 -10.52115039
   -9.87091057 -10.58490301 -10.27440591 -10.81017736 -10.33126127
  -10.74076384 -10.46931756 -11.22517639 -10.25273763 -10.76911251
  -10.52358783 -10.25473782 -11.06163661  -9.94469841 -10.60804493
  -10.51612902 -10.52615638  -9.99063433  -9.65927726 -10.56291229
  -10.28337559 -10.52136754  -9.8158606  -10.78848654 -10.6112873
   -9.81187106 -10.26546674 -10.49838032 -10.4402717  -10.1855070

In [12]:

#coordinate descent algorithm to optimize beta
def solve_beta(X, W, d, n, a, lbd, L, max_iter, converge):
  W1 = np.repeat(W[...,np.newaxis], n, axis=1)
  W1 = W1 - W1.T

  beta = np.zeros(d)
  j0 = 16
  beta[j0] = 1


  old_beta = np.copy(beta)
  for j in np.delete(range(d),j0):
      row = X[j,:]
      Xbeta = np.matmul(beta,X)
      grad = g(Xbeta,W1,row,a,n)
      beta[j] = Soft_thres(beta[j]-grad/L, lbd/L)

  i=1
  while (i<max_iter): 

      old_beta = np.copy(beta)
      i = i+1
      for j in np.delete(range(d),j0):
          row = X[j,:]
          Xbeta = np.matmul(beta,X)
          grad = g(Xbeta,W1,row,a,n)
          beta[j] = Soft_thres(beta[j]-grad/L, lbd/L)

      print('l2 norm of diff', np.linalg.norm(beta-old_beta))
      if (np.linalg.norm(beta-old_beta)<converge) and (i>9):
        break
  
  
      
  print('iter num', i)
  return(beta)



In [55]:
from sklearn.preprocessing import normalize
import time 
start=time.time()

d=305
n=319


#cross validation
cv_fold=2
ncv=int(n*(cv_fold-1)/cv_fold)
cvindex=np.array([i for i in range(318)])
np.random.shuffle(cvindex)
cvindex = cvindex.reshape(318//cv_fold,cv_fold)


if True:
  X = dat.values[:,3:308].T
  X = normalize(X, axis = 1)*np.sqrt(n)
  A_raw = dat.values[:,1]
  A = np.zeros(n)
  A[A_raw=="BUP"]=1

  Y=-dat.values[:,2]
  v=np.mean(Y[A==0])
  pi = sum(A)/n
  W=(Y-v)*(A-pi)/(pi*(1-pi))

  #cross validation
  numl=3
  numa=2
  cvresult=np.zeros((numl,numa))
  for l1 in range(numl):
    for a1 in range(numa):
      cvvalue=0
      for t1 in range(cv_fold):
        Xcv=X[:,cvindex[:,np.where(np.array([i0 for i0 in range(cv_fold)])!=t1)[0]].reshape(-1)]
        Wcv=W[cvindex[:,np.where(np.array([i0 for i0 in range(cv_fold)])!=t1)[0]].reshape(-1)]
        adjustment = np.sqrt(np.sqrt(ncv/np.log(d)))
        a_cv = adjustment*alpha_grid(a1)
        lbd_cv = lambda_grid(l1)*a_cv*np.sqrt(np.log(d)/ncv)
        beta_cv = solve_beta(Xcv,Wcv,d,ncv,a=a_cv,lbd=lbd_cv,L=5*a_cv,max_iter=20,converge=np.sqrt(d)*1e-4)

        #calculate cv value
        Xbeta_cv = np.matmul(beta_cv,X[:,cvindex[:,t1]])
        candidate = list(Xbeta_cv)
        candidate.append(min(Xbeta_cv)-0.01)

        c_cv_objective = list(map(lambda x: ipw(Y1=Y[cvindex[:,t1]],
                                           A1=A[cvindex[:,t1]],xbeta1=Xbeta_cv,c1=x), candidate)) 
        #print('cv_obj', max(c_cv_objective))

        cvvalue = cvvalue+max(c_cv_objective)

      cvresult[l1,a1]=cvvalue
    
  
  print(cvresult)
  max_index = np.argmax(cvresult)
  l1=max_index//numa
  a1=max_index%numa
  print(l1,a1)

  adjustment = np.sqrt(np.sqrt(n/np.log(d)))
  a=adjustment*alpha_grid(a1)
  lbd=lambda_grid(l1)*a*np.sqrt(np.log(d)/n)

  print('lambda= ',lbd, 'alpha=',a)
  beta = solve_beta(X,W,d,n,a=a,lbd=lbd,L=5*a,max_iter=100,converge=np.sqrt(d)*1e-4)
  print('nonzero beta=',sum(beta>0))
  
  #estimate threshold
  Xbeta=np.matmul(beta,X)
  candidate = list(Xbeta)
  candidate.append(min(Xbeta)-0.01)
  c_cv_objective = list(map(lambda x: h(W, Xbeta, x), candidate)) 
  max_index = np.argmax(c_cv_objective)
  b0= candidate[max_index]
  print('threshold=',b0)

  sample1=set(np.where(Xbeta>b0)[0])
  sample0=set(np.where(Xbeta<=b0)[0])
  bup=set(np.where(A==1)[0])
  ser=set(np.where(A==0)[0])
  print('treatment regimes:')
  print(len(sample1&bup))
  print(len(sample1&ser))
  print(len(sample0&bup))
  print(len(sample0&ser))

  #Estimated Value
  valid_samples = set((sample1&bup)|(sample0&ser))
  print('len of overlap', len(valid_samples))
  bootmean=np.zeros(1000)
  bootn=1000
  for i in range(1000): 
      boot= np.random.choice([i for i in range(n)],bootn)
      boot_valid = list(map(lambda x: x in valid_samples, boot))
      IPW=Y[boot]*boot_valid/(A[boot]*pi+(1-A[boot])*(1-pi))
      bootmean[i]=sum(IPW)/bootn

  print('mean', np.mean(bootmean))
  print('lower', np.sort(bootmean)[25])
  print('upper', np.sort(bootmean)[975])
  


print('time cost:', time.time()-start)

l2 norm of diff 0.1662746087615058
l2 norm of diff 0.10500306812055941
l2 norm of diff 0.07539840347038075
l2 norm of diff 0.05582744767211317
l2 norm of diff 0.04234866648029802
l2 norm of diff 0.03381577094920187
l2 norm of diff 0.028774954032474494
l2 norm of diff 0.0255328429912962
l2 norm of diff 0.02264150867697554
l2 norm of diff 0.02036404576274343
l2 norm of diff 0.01798738092898798
l2 norm of diff 0.01642534802977618
l2 norm of diff 0.015129138944055836
l2 norm of diff 0.013948114661525577
l2 norm of diff 0.012987240952356333
l2 norm of diff 0.011427759909365391
l2 norm of diff 0.010294727815373604
l2 norm of diff 0.009595040604025216
l2 norm of diff 0.008842751435706401
iter num 20
l2 norm of diff 0.1941821890706094
l2 norm of diff 0.11479579554317874
l2 norm of diff 0.08163387341356365
l2 norm of diff 0.06527426630974648
l2 norm of diff 0.052298591058049286
l2 norm of diff 0.04464281832308861
l2 norm of diff 0.038501970788942
l2 norm of diff 0.033539850294192174
l2 norm of 